In [2]:
import csv
from getpass import getpass
from time import sleep
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

def get_info_tweet(tweet):
    try:
        post_date = tweet.find_element_by_xpath('.//time').get_attribute('datetime')
    except NoSuchElementException:
        return
    comment = tweet.find_element_by_xpath('.//div[2]/div[2]/div[1]').text
    responding = tweet.find_element_by_xpath('.//div[2]/div[2]/div[2]').text
    text = comment + responding
    reply = tweet.find_element_by_xpath('.//div[@data-testid="reply"]').text
    retweet = tweet.find_element_by_xpath('.//div[@data-testid="retweet"]').text
    like = tweet.find_element_by_xpath('.//div[@data-testid="like"]').text
    
    tweet_info = (post_date, text, reply, retweet, like)
    return tweet_info

driver = webdriver.Chrome(ChromeDriverManager().install())

driver.get('https://www.twitter.com/login')

sleep(5)

username = driver.find_element_by_xpath('//input[@name="session[username_or_email]"]')
username.send_keys('damoralesr97')
my_password = getpass()

password = driver.find_element_by_xpath('//input[@name="session[password]"]')
password.send_keys(my_password)
password.send_keys(Keys.RETURN)

sleep(5)

search_input = driver.find_element_by_xpath('//input[@aria-label="Search query"]')
search_input.send_keys('covid')
search_input.send_keys(Keys.RETURN)

sleep(5)

data = []
tweet_ids = set()
last_position = driver.execute_script("return window.pageYOffset;")
scrolling = True

while scrolling:
    page_cards = driver.find_elements_by_xpath('.//div[@data-testid="tweet"]')
    for card in page_cards[-15:]:
        tweet = get_info_tweet(card)
        if tweet:
            tweet_id = ''.join(tweet)
            if tweet_id not in tweet_ids:
                tweet_ids.add(tweet_id)
                data.append(tweet)
    
    scroll_attempt = 0
    while True:
        driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
        sleep(1)
        curr_position = driver.execute_script("return window.pageYOffset;")
        if last_position == curr_position:
            scroll_attempt += 1
            
            if scroll_attempt >= 3:
                scrolling = False
                break
            else:
                sleep(2)
        else:
            last_position = curr_position
            break
            
with open('tweets.csv', 'w', newline='', encoding='utf-8') as doc:
    header = ['Fecha', 'abstract', 'reply', 'retweet', 'like']
    writer = csv.writer(doc)
    writer.writerow(header)
    writer.writerows(data)



====== WebDriver manager ======
Current google-chrome version is 92.0.4515
Get LATEST driver version for 92.0.4515
Driver [/Users/damoralesr97/.wdm/drivers/chromedriver/mac64/92.0.4515.107/chromedriver] found in cache


········
